# Box-constrained MAF sampling and densities

- simple (scaled) logistic transformation of the output variables of a (conditional!) MAF ensures boundedness of the density support and hence agreement with uniform priors (with rectangular support).
- kind of a very tailored solutions, but uniform priors  appear to be *really* popular in the field of likelihood-free posterior inference.

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
import timeit

from delfi.utils.viz import plot_pdf
import delfi.inference as infer
import delfi.distribution as dd

from lfimodels.snl_exps.util import load_setup_gauss as load_setup

# simulation setup
setup_dict = load_setup() # get some sensible values for the MAF as used for the SNPE-C project

seed=43

# some convention adjustments across code packages (model inputs)
kwargs = {}

kwargs['n_hiddens'] = setup_dict['n_hiddens']
kwargs['act_fun'] = setup_dict['act_fun']
kwargs['n_mades'] = setup_dict['n_mades']

kwargs['n_inputs'] = (3,)
kwargs['n_outputs'] = 3

kwargs['batch_norm'] = setup_dict['batch_norm']
assert 'batch_norm' in kwargs.keys() and not kwargs['batch_norm']
kwargs['rng'] = np.random.RandomState(seed=seed)

assert len(kwargs['n_inputs']) == 1, 'only vector-shaped inputs!'
kwargs['n_inputs'] = kwargs['n_inputs'][0]

kwargs['upper'] =  3.
kwargs['lower'] = -3.




In [ ]:
from delfi.utils.BoxConstraintConditionalAutoregressiveFlow import BoxConstraintConditionalAutoregressiveFlow
 
maf = BoxConstraintConditionalAutoregressiveFlow(**kwargs)

# check 2D (against pdf)

In [ ]:
assert kwargs['n_outputs'] == 2

resolution, eps = 50, 1e-5
x_grid = np.linspace(kwargs['lower']+eps, kwargs['upper']-eps, resolution)
x_,y_ = np.meshgrid(x_grid, x_grid)
thetas = np.concatenate([x_.reshape([-1, 1]), y_.reshape([-1, 1])], axis=1)

plt.figure(figsize=(10,15))
for k in range(3):

    xo = np.random.normal(size=3)
    samples = maf.gen(x=xo, n_samples=100000)

    xy = (np.tile(xo, (thetas.shape[0], 1)), 
          thetas)

    pp = np.zeros(xy[0].shape[0])
    for i in range(pp.size):
        pp[i] = maf.eval((xy[0][i],xy[1][i]), log=False)
    pp = pp.reshape(resolution, resolution)

    plt.subplot(3,2,1+2*k)
    plt.imshow(pp, origin='lower', interpolation='none')

    prob = pp.sum() * maf.diff**2 / resolution**2
    plt.ylabel('numerically integrated mass within box : ' + str(np.round(prob*100)/100))
    plt.title('MAF density')
    
    plt.subplot(3,2,2+2*k)
    plt.hist2d(samples[:,0], samples[:,1], bins=np.linspace(kwargs['lower'], kwargs['upper'], resolution))
    plt.title('sample histogram')

#plt.savefig('/home/mackelab/Desktop/boxconstraint_mafs_density_vs_samples.pdf')

plt.show()



# check 3D (only samples)

In [ ]:
assert kwargs['n_outputs'] == 3 # else need different grid...
resolution, eps = 50, 1e-5
x_grid = np.linspace(kwargs['lower']+eps, kwargs['upper']-eps, resolution)
x_,y_,z_ = np.meshgrid(x_grid, x_grid, x_grid)
thetas = np.concatenate([x_.reshape([-1, 1]), y_.reshape([-1, 1]), z_.reshape([-1, 1])], axis=1)

xo = np.random.normal(size=kwargs['n_inputs'] )
samples = maf.gen(x=xo, n_samples=100000)

plot_pdf(dd.Gaussian(m=0.00000123*np.ones(kwargs['n_outputs']), S=1e-30*np.eye(kwargs['n_outputs'])), 
         samples=samples.T,
         #lims=[[-2,5],[-2,5],[-2,5]], # lims should be within 'upper' and 'lower' !!
         resolution=100,
         ticks=True,
         figsize=(16,16));

